In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import scipy as sp
import math
from numpy import sin,cos,pi
plt.style.use('seaborn')
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

from sigproc import normaliseData
from sigproc import encodeActivityName
from sigproc import segmentation 
from sigproc import getInputVector

In [2]:
dance_moves = ["dab", "elbowkick", "gun", "hair", "listen", "logout", "pointhigh", "sidepump", "wipetable"]
dance_moves

['dab',
 'elbowkick',
 'gun',
 'hair',
 'listen',
 'logout',
 'pointhigh',
 'sidepump',
 'wipetable']

In [3]:
sub = 1
num = 1
countRows = 0
frames = []
while sub <=7:
    for move in dance_moves:
        while num <=3:
            location = f"../data1/subject{sub}/{move}{num}.csv"
            df = pd.read_csv(location, index_col=None) 
            df.drop("Unnamed: 0",axis = 1, inplace = True)
            df.dropna(inplace= True)
            df["subject"] = sub
            df["trial"] = num
            df["target"] = move
            df.reset_index()
            frames.append(df)
            # print(df)
            countRows += (df.shape[0])
            # print("\n")
            # print(f"As of last read at {location}, {countRows} rows of data were cumulated")
            # print("\n")
            num+=1
        num = 1
    sub+=1
      
rawdata = pd.concat(frames, axis=0, ignore_index=True)

In [4]:
rawdata.head()

,yaw,pitch,roll,gx,gy,gz,ax,ay,az,subject,trial,target
0,45.98,6.60,-1.07,-10,-4,-3,-1023,481,181,1,1,dab
1,0.01,-0.01,0.33,-1,0,0,18,-16,-1421,1,1,dab
2,0.01,-0.02,0.29,-2,0,0,0,11,-370,1,1,dab
3,0.03,-0.02,0.21,-2,-1,-1,28,16,-41,1,1,dab
4,0.06,-0.01,0.15,-1,-1,-1,8,4,58,1,1,dab


In [5]:
rawdata.tail()

,yaw,pitch,roll,gx,gy,gz,ax,ay,az,subject,trial,target
310191,-78.17,-18.74,-13.66,-38,-58,-98,1386,-2578,2648,7,3,wipetable
310192,-73.57,-13.30,-17.93,-46,-79,-81,235,-4201,3158,7,3,wipetable
310193,-70.85,-6.83,-21.74,-29,-116,-47,-1613,-6574,4646,7,3,wipetable
310194,-71.99,-1.02,-24.06,-25,-110,7,-3173,-7435,6233,7,3,wipetable
310195,-77.04,1.94,-25.96,-8,-105,80,-5552,-7294,8005,7,3,wipetable


In [6]:
countRows

310196

In [7]:
rawdata.describe()

,yaw,pitch,roll,gx,gy,gz,ax,ay,az,subject,trial
count,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000
mean,-46.726043,-9.851313,-15.575414,1.497850,-3.546132,-1.551210,-3860.604921,-625.942362,4343.535723,4.012341,1.997811
std,71.660987,34.015990,31.106017,62.234371,69.444694,115.890831,3810.411576,3154.165889,2978.299761,1.991473,0.816167
min,-180.000000,-179.950000,-176.410000,-693.000000,-401.000000,-605.000000,-20182.000000,-19924.000000,-12047.000000,1.000000,1.000000
25%,-98.200000,-33.790000,-35.350000,-28.000000,-34.000000,-48.000000,-6250.000000,-2186.000000,2155.000000,2.000000,1.000000
50%,-48.535000,-16.630000,-15.020000,1.000000,-2.000000,1.000000,-4315.000000,-298.000000,4372.000000,4.000000,2.000000
75%,-0.320000,19.400000,1.270000,32.000000,23.000000,49.000000,-1964.000000,1216.000000,6608.000000,6.000000,3.000000
max,180.000000,179.950000,177.980000,538.000000,486.000000,650.000000,24531.000000,672320.000000,13315.000000,7.000000,3.000000


In [8]:
map_num_to_dance = {}
map_dance_to_num = {}
for i,dance in enumerate(dance_moves):
    map_num_to_dance[i] = dance
    map_dance_to_num[dance] = i

map_num_to_dance
map_dance_to_num

{'dab': 0,
 'elbowkick': 1,
 'gun': 2,
 'hair': 3,
 'listen': 4,
 'logout': 5,
 'pointhigh': 6,
 'sidepump': 7,
 'wipetable': 8}

In [9]:
y = []
for target in rawdata["target"]:
    y.append(map_dance_to_num[target])

Y = np.asarray(y)

In [10]:
Y

array([0, 0, 0, ..., 8, 8, 8])

In [11]:
Y.max()

8

In [12]:
X = rawdata[rawdata.columns.difference(["subject", "trial", "target"])]

In [13]:
X.head()

,ax,ay,az,gx,gy,gz,pitch,roll,yaw
0,-1023,481,181,-10,-4,-3,6.60,-1.07,45.98
1,18,-16,-1421,-1,0,0,-0.01,0.33,0.01
2,0,11,-370,-2,0,0,-0.02,0.29,0.01
3,28,16,-41,-2,-1,-1,-0.02,0.21,0.03
4,8,4,58,-1,-1,-1,-0.01,0.15,0.06


In [14]:
normaliseData(X, X.columns)

In [15]:
X.head()

,ax,ay,az,gx,gy,gz,pitch,roll,yaw
0,-0.0417,0.0007,0.0136,-0.0186,-0.0082,-0.0046,0.0367,-0.0060,0.2554
1,0.0007,-0.0000,-0.1067,-0.0019,0.0000,0.0000,-0.0001,0.0019,0.0001
2,0.0000,0.0000,-0.0278,-0.0037,0.0000,0.0000,-0.0001,0.0016,0.0001
3,0.0011,0.0000,-0.0031,-0.0037,-0.0021,-0.0015,-0.0001,0.0012,0.0002
4,0.0003,0.0000,0.0044,-0.0019,-0.0021,-0.0015,-0.0001,0.0008,0.0003


In [16]:
encodeActivityName(rawdata, "target", "encodedTarget")

9
['dab', 'elbowkick', 'gun', 'hair', 'listen', 'logout', 'pointhigh', 'sidepump', 'wipetable']


In [17]:
rawdata.tail()

,yaw,pitch,roll,gx,gy,gz,ax,ay,az,subject,trial,target,encodedTarget
310191,-78.17,-18.74,-13.66,-38,-58,-98,1386,-2578,2648,7,3,wipetable,8
310192,-73.57,-13.30,-17.93,-46,-79,-81,235,-4201,3158,7,3,wipetable,8
310193,-70.85,-6.83,-21.74,-29,-116,-47,-1613,-6574,4646,7,3,wipetable,8
310194,-71.99,-1.02,-24.06,-25,-110,7,-3173,-7435,6233,7,3,wipetable,8
310195,-77.04,1.94,-25.96,-8,-105,80,-5552,-7294,8005,7,3,wipetable,8


In [18]:
data, lbl = segmentation(rawdata, 50, 2,50,"encodedTarget")

In [19]:
data.shape

(6202, 100, 6)

In [20]:
lbl.shape

(6202,)

In [21]:
x_windows = getInputVector(data, 50, 2, 6)

In [22]:
x_windows.shape

(6202, 600)

In [23]:
x_windows[0]

array([-1.0230e+03,  1.8000e+01,  0.0000e+00,  2.8000e+01,  8.0000e+00,
        3.0000e+00, -4.0000e+01, -1.6000e+01, -2.3000e+01, -1.8000e+01,
       -5.9000e+01, -4.7000e+01, -8.0000e+01, -2.1000e+02, -4.4900e+02,
       -9.5200e+02, -1.1230e+03, -2.5390e+03, -4.0690e+03, -4.1000e+03,
       -1.8160e+03,  3.4200e+02,  8.1100e+02,  6.9300e+02, -1.6600e+02,
       -1.4900e+03, -7.4900e+02, -5.2350e+03, -1.3456e+04, -1.6031e+04,
       -1.4936e+04, -1.3469e+04, -1.2927e+04, -1.1997e+04, -1.1222e+04,
       -1.0661e+04, -8.5790e+03, -7.2190e+03, -7.2770e+03, -7.6700e+03,
       -7.9470e+03, -7.9700e+03, -6.9000e+03, -7.3350e+03, -5.7110e+03,
       -7.6280e+03, -5.7650e+03, -6.7550e+03, -6.6700e+03, -6.8000e+03,
       -6.4360e+03, -6.1080e+03, -5.4270e+03, -5.0510e+03, -4.5230e+03,
       -3.0030e+03, -8.5800e+02,  1.9830e+03,  4.2370e+03,  5.9900e+03,
        7.8290e+03,  1.0610e+04,  1.1202e+04,  9.8600e+03,  7.3970e+03,
        3.6160e+03,  3.3800e+02, -1.3610e+03, -4.4270e+03, -6.35

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [25]:
 X_train, X_test, y_train, y_test = train_test_split(x_windows, lbl, test_size=0.33, random_state=42)

In [40]:
knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train, y_train)

KNeighborsClassifier()

In [41]:
y_preds = knn_clf.predict(X_test)

In [45]:
print("KNN Confusion Matrix:")
print("\n")
print(confusion_matrix(y_test, y_preds))
print("\n")
print("KNN Classification Report:",)
print("\n")
print (classification_report(y_test, y_preds))
print("\n")
print("KNN Accuracy:", accuracy_score(y_test,y_preds))
print("\n")

KNN Confusion Matrix:


[[210   2   0   0   0   8   4   0   0]
 [  0 218   4   0   4   0   0   0   0]
 [  0   5 197   0   1   2   0   0   0]
 [  0   3   3 219   9   0   0   0   0]
 [  0   1   2   2 228   0   0   0   0]
 [  0   1   0   0   0 238   0   0   0]
 [  0   0   0   2   7   0 210   0   0]
 [  0   2   4   2   4   2   1 224   0]
 [  1   0   0   1   2   3   0   1 220]]


KNN Classification Report:


              precision    recall  f1-score   support

           0       1.00      0.94      0.97       224
           1       0.94      0.96      0.95       226
           2       0.94      0.96      0.95       205
           3       0.97      0.94      0.95       234
           4       0.89      0.98      0.93       233
           5       0.94      1.00      0.97       239
           6       0.98      0.96      0.97       219
           7       1.00      0.94      0.97       239
           8       1.00      0.96      0.98       228

    accuracy                           0.96      20

In [33]:
rawdata["az"][::-1]

310195    8005
310194    6233
310193    4646
310192    3158
310191    2648
          ... 
4           58
3          -41
2         -370
1        -1421
0          181
Name: az, Length: 310196, dtype: int64